This is a work-in-progress proof-of-concept for 3D.

In [ ]:
from jupyter_widget_engine import JupyterWidgetEngine
import random
import numpy as np
import copy

L = 22 # screen size
dist = 32 # draw distance
h0 = 2 # player height


def get_height(x,z,block=20,road=4,verge=4,height=8):
    
    if -x%block in range(road):# or z%block in range(road):
        h = 1
        color = 'grey'
    elif -x%block in range(road+verge,block-verge) and z%block in range(road+verge,block-verge):
        h = height
        color = 'orange'
    else:
        h = 1
        color = 'green'
    return h, color

                        
def start(engine):
    
    # make parameters into attribites of the engine object
    engine.dist = dist
    engine.h0 = h0
    
    x0 = 0
    z0 = 0
    engine.pos = [x0,height[x0,z0],z0]
    
    # initialize by calling the next frame
    engine.next_frame(engine)
    
    
def next_frame(engine):
    
    L = engine.L 
    
    # x,y,z is position of feet
    # x is left/right, y is up/down, z is forwards/backwards
    [x,y,z] = engine.pos

    # x,y+h0,z is position of eyes
    h0 = engine.h0

    view = {}
    # imagine a 1x1 frame centered on x,y,z+1
    # we define a coorinate system with origin at the top left
    # coordinates are scaled such that bottom right is (L,L)
    # X,Y is position that an object appears in this frame
    for X in range(L):
        for Y in range(L):
            # dx,dy,dz is displacement from eyes to something the player is looking at
            dz = 0
            # extend dz until we find the surface (with the draw distance)
            while dz<engine.dist and (X,Y) not in view:
                # dx,dy,dz is displacement from eyes to something the player is looking at
                dx = (X/L - 0.5)*dz
                dy = -(Y/L - 0.5)*dz
                # distance from eyes
                d = (dx**2+dy**2+dz**2)**0.5
                # xx,yy,zz is position of point that the player is looking at
                xx = int(x+dx)
                zz = int(z+dz)  
                yy = y+h0+dy
                # once we find the surface...
                h,color = get_height(xx,zz)
                if h>yy:
                    # ...record height, distance and color
                    view[X,Y] = (h,d,color)
                dz += 1
             
    for X in range(L):
        for Y in range(L):
            if (X,Y) in view:
                (h,d,color) = view[X,Y]     
                engine.screen[X,Y].set_color(color)
                if color in ['blue']:
                    engine.screen[X,Y].set_brightness( False )
                else:
                    engine.screen[X,Y].set_brightness( d>engine.dist/2 )
            else:
                engine.screen[X,Y].set_color('blue')
                engine.screen[X,Y].set_brightness( True )
                
    engine.screen['text'].set_text('Position: ' +str((x,y,z)))
                
    # change position based on controller input
    # if the new position is on a grass tile of the current screen, undo the movement
    if engine.controller['up'].value:
        z+=1
    if engine.controller['down'].value:
        z-=1
    if engine.controller['left'].value:
        x-=1
    if engine.controller['right'].value:
        x+=1
        
    y,_ = get_height(x,z)
    engine.pos = [x,y,z]
                        

engine = JupyterWidgetEngine(start,next_frame,L=L)